<a href="https://colab.research.google.com/github/grfaith/AmStory/blob/main/Finding_explore_kw_proximity_hits_in_AS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###Installs

!pip install datasets
!pip install ipympl


This is code for looking at a range of years in the American Stories data and finding articles which appear with the string 'explor' and saving their information to disk.

In [ ]:
#Imports
import json
import pandas as pd
from datasets import load_dataset
import tqdm as tq
from google.colab import files
import re

kw_distance = 150
start_year = 1929
end_year = 1940 #This is end-year exclusive (I think)

In [4]:
### Cell for loading files from local drive
kw_file = files.upload()

# Specify custom column names
column_names = ["kword", "kwyear"]

# Read the uploaded CSV file into a DataFrame with custom column names
for fn in kw_file.keys():
    kw_df = pd.read_csv(fn, names=column_names, header=None)

# Display information about the uploaded file
for fn in kw_file.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
          name=fn, length=len(kw_file[fn])))



Saving AS_Explore_KW.csv to AS_Explore_KW.csv
User uploaded file "AS_Explore_KW.csv" with length 265 bytes


In [5]:
# Defining function to load dataset

def load_text_dataset(dataset_year_str):
    """
    This function pulls a dataset of a specific year from the HuggingFace Hub.

    Parameters:
        dataset_year (int): The year of the dataset to be pulled..

    Returns:
        dataset_article_level: dataset for appropriate year
    """
    # Download data for the dataset year at the associated article level (Default)
    # dataset = load_dataset("dell-research-harvard/AmericanStories", "subset_years", year_list=[dataset_year])

    # now let's load our data, we have to specify the huggingface location of our
    # data, the fact that we want to have a subset of years, and our desired years
    dataset_article_level=load_dataset("dell-research-harvard/AmericanStories",
                                      "subset_years",
                                       year_list=[dataset_year_str],
                                       trust_remote_code=True
                                       )

    return dataset_article_level

In [6]:
### Function to filter kw data in use based on years of discovery in kw file.

def get_kw(dataset_year_str):
    """
    This function loads a CSV file to create a DataFrame and filters out keywords where the second column is less than 1774
    Parameters:
        kw_file loaded from prompt above
    Returns:
        pandas.DataFrame: The filtered Data
    """
    # Convert dataset_year to integer
    dataset_year = int(dataset_year_str)

    # Filter the rows based on the condition
    kw_df_filter = kw_df[kw_df['kwyear'] <= dataset_year]

    # print(kw_df_filter)

    return kw_df_filter



In [7]:

def process_kw(dataset_year_str, kw_df_filter, dataset_article_level):
    """
    This function processes words in a DataFrame.

    Parameters:
        kw_df_filter (pandas.DataFrame): The DataFrame containing keywords
        dataset_article (DatasetDict): A dictionary-like object containing datasets for different years.

    Returns:
        dataset_year_hits (DataFrame or None): A DataFrame containing results if found, or None if no results were found.
    """
    print ("Searching within ", dataset_year_str)

    # Creating an empty dataframe
    current_year_df = pd.DataFrame()

    for index, row in kw_df_filter.iterrows():
        explore_kw = row.iloc[0]
        result_df = kw_pair_search(dataset_article_level, dataset_year_str, explore_kw, kw_distance)
        # Concatenate the single search result onto the results DataFrame
        current_year_df = pd.concat([current_year_df, result_df], ignore_index=True)

    return current_year_df


In [8]:
## key word search within articles to find keywords with kw_distance of the string 'explor'

def kw_pair_search(dataset_article, dataset_year, explor_kw, kw_distance):
    """
    This function searches through the dataset, by kw, to look for matching articles.

    Parameters:
        dataset_article (DatasetDict): A dictionary-like object containing datasets for different years.
        dataset_year (int): The year of the dataset.
        explor_kw (str): The keyword to search for.
        kw_distance (int): The maximum distance allowed between occurrences of the two keywords.

    Returns:
        df_of_articles_containing_two_words (DataFrame): A DataFrame containing article IDs and article texts of articles containing both keywords within the specified distance.
    """
    # Access the dataset for the specific year
    year_dataset = dataset_article[dataset_year]

    # Access the 'raw_data_string' column
    articles = year_dataset['article']

    # Create empty list to store matching articles
    articles_containing_two_words = []

    for article_n, article_text in enumerate(articles):
        article_text = article_text.lower()
        if "explor" in article_text and explor_kw in article_text:
            # print ("We have a hit at", article_n)

            # Define the pattern to search for "explor" within words
            pattern_explor = re.compile(r'\b\w*explor\w*\b')

            # Find indices of matches in the article text
            explor_indices = [match.start() for match in pattern_explor.finditer(article_text)]

            # Define the pattern to search for the value of 'explor_kw' within words
            pattern_kw = re.compile(r'\b\w*' + re.escape(explor_kw) + r'\w*\b', flags=re.IGNORECASE)

            # Find indices of matches in the article text
            kw_indices = [match.start() for match in pattern_kw.finditer(article_text)]


            # Check if there are any pairs of indices within the specified distance
            for explor_index in explor_indices:
                for kw_index in kw_indices:
                    if abs(explor_index - kw_index) <= kw_distance:
                        # print ("SUCCESS!!", dataset_year, article_n)
                        # Append the matching article information to the list
                        articles_containing_two_words.append({
                            'article_year': dataset_year,
                            'keyword_hit': explor_kw,
                            'row_number': article_n,
                            'article_ID': dataset_article[dataset_year][article_n]["article_id"],
                            'newspaper_name': dataset_article[dataset_year][article_n]["newspaper_name"],
                            'edition': dataset_article[dataset_year][article_n]["edition"],
                            'date': dataset_article[dataset_year][article_n]["date"],
                            'page': dataset_article[dataset_year][article_n]["page"],
                            'headline': dataset_article[dataset_year][article_n]["headline"],
                            'byline': dataset_article[dataset_year][article_n]["byline"],
                            'article': dataset_article[dataset_year][article_n]["article"],
                        })
                        # Once a matching pair is found, break out of the loop
                        break
                    else:
                        continue

    # Convert the list of dictionaries to a DataFrame
    df_of_articles_containing_two_words = pd.DataFrame(articles_containing_two_words)
    return df_of_articles_containing_two_words


In [9]:
results = pd.DataFrame()

for loop_year in range(start_year, end_year):
    dataset_year_str = str(loop_year)
    try:
        # Load dataset for current loop_year
        dataset_article_level = load_text_dataset(dataset_year_str)

        # Get valid keyword filters for current year
        kw_df_filter = get_kw(loop_year)
        # print ("Processing year", dataset_year_str)

        # Process keyword filters and dataset for current loop year to get hits
        year_search_result = process_kw(dataset_year_str,kw_df_filter, dataset_article_level)

        # if not year_search_result.empty:
        #   print("Found: ", year_search_result)
        # else:
        #   print("No results found for ", dataset_year_str)

        # Concatenate the single search result onto the results DataFrame
        results = pd.concat([results, year_search_result])

    except ValueError:
        print(f"Dataset empty for {dataset_year_str}. Moving to the next year.")
        continue

results = results.reset_index(drop=True)
results.to_csv('AS_Explor_KW_Hits_Raw.csv', index=False)
files.download("AS_Explor_KW_Hits_Raw.csv")
print("Finished")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Only taking a subset of years. Change name to 'all_years' to use all years in the dataset.
{'1929': 'https://huggingface.co/datasets/dell-research-harvard/AmericanStories/resolve/main/faro_1929.tar.gz'}


Generating 1929 split: 0 examples [00:00, ? examples/s]

Loading associated
Searching within  1929
Only taking a subset of years. Change name to 'all_years' to use all years in the dataset.
{'1930': 'https://huggingface.co/datasets/dell-research-harvard/AmericanStories/resolve/main/faro_1930.tar.gz'}


Generating 1930 split: 0 examples [00:00, ? examples/s]

...

Generating 1939 split: 0 examples [00:00, ? examples/s]

Loading associated
Searching within  1939


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Finished


In [11]:
# Run this cell if you have to interrupt the above loop to save progress.
#  Update file names as appropriate.


results = results.reset_index(drop=True)
results.to_csv('AS_Explor_KW_Hits_Prox_Raw_Part_3.csv', index=False)
files.download("AS_Explor_KW_Hits_Prox_Raw_Part_3.csv")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print ("Now save output file to local disk")
input("Press Enter to continue...")


# **SOLVENT FRONT**

In [10]:
print(results.info)


<bound method DataFrame.info of      article_year keyword_hit  row_number  \
0            1929    astronom      154508   
1            1929    herschel       29129   
2            1929        mars       35835   
3            1929        mars       46397   
4            1929        mars       46397   
...           ...         ...         ...   
1205         1939        star      536145   
1206         1939        star      536145   
1207         1939   telescope      284495   
1208         1939   telescope      381312   
1209         1939   telescope      426437   

                                             article_ID  \
0     11_1929-12-15_p47_sn83045462_00280600143_19291...   
1     2_1929-06-08_p5_sn87062013_00414185113_1929060...   
2     1_1929-10-12_p1_sn87062013_00414185113_1929101...   
3     3_1929-09-24_p2_sn83045499_00514159555_1929092...   
4     3_1929-09-24_p2_sn83045499_00514159555_1929092...   
...                                                 ...   
1205  17_1939-

In [ ]:
# let's start with deciding which years we want data for
scan_level_desired_years = ["1900",]

# now let's load our data, we have to specify the huggingface location of our
# data, the fact that we want to have a subset of years, and our desired years
dataset_scan_level=load_dataset("dell-research-harvard/AmericanStories",
                                "subset_years_content_regions",
                                year_list=scan_level_desired_years
                                )

In [ ]:
print(dataset_article_level['1900'][377385]["article"])